In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

from numpy import vectorize

import GCRCatalogs
from GCR import GCRQuery

In [2]:
print('GCRCatalogs =', GCRCatalogs.__version__, '|' ,'GCR =', GCRCatalogs.GCR.__version__)

GCRCatalogs = 0.10.0 | GCR = 0.8.7


In [3]:
bin_edges = np.concatenate(
    [np.arange(0.60, 1.95, 0.1),
     [2.02, 2.22, 2.34, 2.52, 2.66,
      2.84, 2.98, 3.16, 3.30, 3.48,
      3.62, 3.78, 3.94, 4.10]])
delta_zs = bin_edges[1:] - bin_edges[:-1]
# Initial density values are in units, N / (deg^2 dz)
# hence multiplying by the redshift bin width.
ELG_density = np.array( # units: N / degree^2
    [ 309, 2269, 1923, 2094, 1441, 1353,  523,  466,
      329,  126,    0,    0, 0,    0,    0,    0,
        0,    0,    0,    0, 0,    0,    0,    0,
        0,    0,    0]) * delta_zs
QSO_density = np.array(# units: N / degree^2
    [47, 55, 61, 67, 72, 76, 80, 83,
     85, 87, 87, 87, 86, 82, 69, 53,
     43, 37, 31, 26, 21, 16, 13,  9,
      7,  5,  3]) * delta_zs

In [4]:
lrg_density = np.loadtxt(
    '/global/projecta/projectdirs/lsst/groups/PZ/PhotoZDC2/protoDC2v5/DESI_reference_redshifts/'
    'lrg_densities_20180725.txt')
lrg_smoothed_bin_edges = np.arange(0.25, 1.26, 0.1)
print(lrg_smoothed_bin_edges)
lrg_smoothed_densities = np.zeros(len(lrg_smoothed_bin_edges) - 1)
for bin_idx in range(10):
    lrg_smoothed_densities[bin_idx] = lrg_density[
        10*bin_idx:10 * (bin_idx + 1), 2].sum()

[0.25 0.35 0.45 0.55 0.65 0.75 0.85 0.95 1.05 1.15 1.25]


In [5]:
catalog = GCRCatalogs.load_catalog('cosmoDC2_v1.1.4_small')

In [6]:
def qsoness(mass, ratio):
    """Simple variable identifying how qso like an object is.
    
    Parameters
    ----------
    mass : `float`
        Black hole mass
    ratio : `float`
        Black hole eddington ratio
        
    Returns
    -------
    qsoness : `float`
        Simple multication of mass * ratio.
    """
    return mass * ratio

In [7]:
catalog.add_derived_quantity('qsoness', qsoness, 'blackHoleMass', 'blackHoleEddingtonRatio')

In [8]:
def create_redshift_stellar_prop_selection(z_bin_mins,
                                           z_bin_maxes,
                                           goal_number_densities,
                                           mag_name='mag_r_lsst',
                                           mag_lim=23.4,
                                           not_columns=None,
                                           selection='qsoness'):
    """Compute star formation rate or stellar mass cuts in redshifts bins given a
    set of bin edges and expected densities.
    
    Paramters
    ---------
    z_bin_mins : `float` array-like
        Minimum bin redshifts.
    z_bin_maxes : `float` array-like
        Maximum bin redshifts.
    goal_number_densities : `float` array-like
        Number of expected objects per redshift bin. Should be in units counts.
    mag_name : `str`
        Name of the magnitude column to cut on.
    mag_lim : `float`
        Maximum value of the magnitude obseved in mag_name.
    not_columns : `dict`
        Dictionary of float arrays from a pervious selection run in this method.
    selection : `string`
        Column to select stellar properites. Valid choices are stellar_mass and
        totalStarFormationRation.
        
    Returns
    -------
    output : `dict`
        Dictionary of bin edges.
    """
    
    columns = [mag_name, 'redshift', selection]
    
    #Pre-cut selections. Magnitude cut avoids duplication of objects from the DESI
    # magnitude limiting cut.
    start_cut = [
        GCRQuery('%s < %.1f' % (mag_name, mag_lim)),
        GCRQuery('redshift > %.2f' % np.min(z_bin_mins)),
        GCRQuery('redshift <= %.2f' % np.max(z_bin_maxes)),
        GCRQuery('mag_r_lsst >= 19.5')
    ]
    if not_columns is not None:
        for not_column in not_columns:
            start_cut.append(GCRQuery('%s' % not_column))

    data = catalog.get_quantities(columns, filters=start_cut)
    
    #Initialize data output dict.
    output = {"z_min": z_bin_mins,
              "z_max": z_bin_maxes,
              "n_goal": goal_number_densities,
              "mag_lim": mag_lim,
              selection: np.full(len(z_bin_maxes), 10 ** 30),
              "mag_name": mag_name}
        
    for bin_idx, (z_min, z_max, n_goal) in enumerate(
            zip(z_bin_mins, z_bin_maxes, goal_number_densities)):
        
        if n_goal <= 0:
            continue

        mask = np.logical_and(data['redshift'] > z_min,
                              data['redshift'] <= z_max)
        
        #If an input dict has been provided, select on totalStarFormationRate
        #as defined by the dictionary. This allows for the removal of objects
        #previously selected from a sample making each selection a set of
        #independent objects.
        stellar_array = data[selection][mask]

        if len(stellar_array) == 0:
            continue
        elif len(stellar_array) < n_goal:
            n_goal = len(stellar_array)
        stellar_array.sort()
        
        output[selection][bin_idx] = stellar_array[-np.int(np.ceil(n_goal))]
            
    return output

In [9]:
qso_dict = create_redshift_stellar_prop_selection(
    bin_edges[:-1],
    bin_edges[1:],
    QSO_density * catalog.sky_area,
    selection='qsoness',
    mag_name="mag_r_lsst",
    mag_lim=23.4)

[<GCR.query.GCRQuery object at 0x2aaae5490c88>, <GCR.query.GCRQuery object at 0x2aaae5490cf8>, <GCR.query.GCRQuery object at 0x2aaae54908d0>, <GCR.query.GCRQuery object at 0x2aaae5490908>]


In [10]:
def not_qsoselection(redshift, mag, mag_r, qsoness):
    """
    """
    mask =  mag_r > 19.5
    mask = np.logical_and(mag < qso_dict['mag_lim'], mask)
    
    z_min = qso_dict['z_min'].min()
    z_max = qso_dict['z_min'].max()
    mask = np.logical_and(mask, redshift > z_min)
    mask = np.logical_and(mask, redshift < z_max)
    
    z_bin_idxes = np.searchsorted(qso_dict['z_min'], redshift) - 1
    
    min_qsoness = qso_dict['qsoness'][z_bin_idxes]
    mask = np.logical_and(qsoness > min_qsoness, mask)
    
    return np.logical_not(mask)

In [11]:
catalog.add_derived_quantity('not_qsoselection',
                             not_qsoselection,
                             'redshift', 'mag_r_lsst', 'mag_r_lsst', 'qsoness')

In [12]:
elg_dict = create_redshift_stellar_prop_selection(
    bin_edges[:-1],
    bin_edges[1:],
    ELG_density * catalog.sky_area,
    not_columns=['not_qsoselection'],
    selection='totalStarFormationRate',
    mag_name="mag_r_lsst",
    mag_lim=23.4)

[<GCR.query.GCRQuery object at 0x2aaae54909e8>, <GCR.query.GCRQuery object at 0x2aaae5490668>, <GCR.query.GCRQuery object at 0x2aaae54900f0>, <GCR.query.GCRQuery object at 0x2aaae54906d8>, <GCR.query.GCRQuery object at 0x2aaae5490a20>]


In [13]:
def not_elgselection(redshift, mag, mag_r, sfr, not_qso):
    """
    """
    mask =  mag_r > 19.5
    mask = np.logical_and(mag < elg_dict['mag_lim'], mask)
    mask = np.logical_and(not_qso, mask)
    
    z_min = elg_dict['z_min'].min()
    z_max = elg_dict['z_min'].max()
    mask = np.logical_and(mask, redshift > z_min)
    mask = np.logical_and(mask, redshift < z_max)
    
    z_bin_idxes = np.searchsorted(elg_dict['z_min'], redshift) - 1
    
    min_sfr = elg_dict['totalStarFormationRate'][z_bin_idxes]
    mask = np.logical_and(sfr > min_sfr, mask)
    
    return np.logical_not(mask)

In [14]:
catalog.del_quantity_modifier('not_elgselection')

In [15]:
catalog.add_derived_quantity('not_elgselection',
                             not_elgselection,
                             'redshift',
                             'mag_r_lsst',
                             'mag_r_lsst',
                             'totalStarFormationRate',
                             'not_qsoselection')

In [16]:
lrg_dict = create_redshift_stellar_prop_selection(
    lrg_smoothed_bin_edges[:-1],
    lrg_smoothed_bin_edges[1:],
    lrg_smoothed_densities * catalog.sky_area,
    not_columns=['not_qsoselection', 'not_elgselection'],
    selection='stellar_mass',
    mag_name="mag_z_lsst",
    mag_lim=23.0)

[<GCR.query.GCRQuery object at 0x2aaae5490240>, <GCR.query.GCRQuery object at 0x2aaae5490080>, <GCR.query.GCRQuery object at 0x2aaae5490c88>, <GCR.query.GCRQuery object at 0x2aaae5490550>, <GCR.query.GCRQuery object at 0x2aaae54909e8>, <GCR.query.GCRQuery object at 0x2aaae54900f0>]


In [21]:
def not_lrgselection(redshift, mag, mag_r, stm, not_qso, not_elg):
    """
    """
    mask =  mag_r > 19.5
    mask = np.logical_and(mag < lrg_dict['mag_lim'], mask)
    mask = np.logical_and(not_qso, mask)
    mask = np.logical_and(not_elg, mask)
    
    z_min = lrg_dict['z_min'].min()
    z_max = lrg_dict['z_min'].max()
    mask = np.logical_and(mask, redshift > z_min)
    mask = np.logical_and(mask, redshift < z_max)
    
    z_bin_idxes = np.searchsorted(lrg_dict['z_min'], redshift) - 1
    
    min_stm = lrg_dict['stellar_mass'][z_bin_idxes]
    mask = np.logical_and(stm > min_stm, mask)
    
    return np.logical_not(mask)

In [28]:
catalog.del_quantity_modifier('not_lrgselection')

In [29]:
catalog.add_derived_quantity('not_lrgselection',
                             not_lrgselection,
                             'redshift',
                             'mag_z_lsst',
                             'mag_r_lsst',
                             'stellar_mass',
                             'not_qsoselection',
                             'not_elgselection')

In [56]:
def magnitude_limit(mag):
    return mag < 19.5

catalog.add_derived_quantity('mag_limit',
                             magnitude_limit,
                             'mag_r_lsst')

In [60]:
pzCalib_iter = catalog.get_quantities(['not_qsoselection', 'not_elgselection',
                                       'not_lrgselection', 'mag_limit'],
                                      return_iterator=True)

In [68]:
for data, (zlo, hpx) in zip(pzCalib_iter, catalog._healpix_files):
    mask = np.logical_not(data['not_qsoselection'])
    np.save('/global/projecta/projectdirs/lsst/groups/PZ/PhotoZDC2/COSMODC2v1.1.4/'
            'PZCalibrateSample_cosmoDC2v1.1.4_small/pzCalibrateQSO_z%i_healpix%i.npy' % (zlo, hpx),
            np.logical_not(data['not_qsoselection']))
    
    mask = np.logical_or(mask, np.logical_not(data['not_elgselection']))
    np.save('/global/projecta/projectdirs/lsst/groups/PZ/PhotoZDC2/COSMODC2v1.1.4/'
            'PZCalibrateSample_cosmoDC2v1.1.4_small/pzCalibrateELG_z%i_healpix%i.npy'  % (zlo, hpx),
            np.logical_not(data['not_elgselection']))

    mask = np.logical_or(mask, np.logical_not(data['not_lrgselection']))
    np.save('/global/projecta/projectdirs/lsst/groups/PZ/PhotoZDC2/COSMODC2v1.1.4/'
            'PZCalibrateSample_cosmoDC2v1.1.4_small/pzCalibrateLRG_z%i_healpix%i.npy'  % (zlo, hpx),
            np.logical_not(data['not_lrgselection']))

    mask = np.logical_or(mask, data['mag_limit'])
    np.save('/global/projecta/projectdirs/lsst/groups/PZ/PhotoZDC2/COSMODC2v1.1.4/'
            'PZCalibrateSample_cosmoDC2v1.1.4_small/pzCalibrateMagLim_z%i_healpix%i.npy'  % (zlo, hpx),
            data['mag_limit'])

    np.save('/global/projecta/projectdirs/lsst/groups/PZ/PhotoZDC2/COSMODC2v1.1.4/'
            'PZCalibrateSample_cosmoDC2v1.1.4_small/pzCalibrateRef_z%i_healpix%i.npy'  % (zlo, hpx),
            mask)

0 10070
0 10071
0 10072
0 10198
0 10199
0 10200
0 10326
0 10327
0 10450
0 9559
0 9686
0 9687
0 9814
0 9815
0 9816
0 9942
0 9943
1 10070
1 10071
1 10072
1 10198
1 10199
1 10200
1 10326
1 10327
1 10450
1 9559
1 9686
1 9687
1 9814
1 9815
1 9816
1 9942
1 9943
2 10070
2 10071
2 10072
2 10198
2 10199
2 10200
2 10326
2 10327
2 10450
2 9559
2 9686
2 9687
2 9814
2 9815
2 9816
2 9942
